## Group proposal
Title: Heart Disease Diagnostic Tool

In [2]:
library(rvest)
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fi

Introduction: 

In [ ]:
set.seed(293)

dataset_test <- download.file(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
    "heart_disease.csv")

heart_data <- read_csv("heart_disease.csv", col_names = FALSE)


heart_data <- rename(heart_data, 
                     age =X1,
                     sex =X2,
                     cp =X3,
                     trestbps =X4,
                     chol = X5,
                     fbs =X6,
                     restecg=X7,
                     thalach=X8,
                     exang=X9,
                     oldpeak=X10,
                     slope=X11,
                     ca=X12,
                     thal=X13,
                     diagnosis=X14)

idx <- heart_data == "?"
is.na(heart_data) <- idx                                 #eliminates ? and replaces with NA

heart_data[] <- sapply(heart_data, as.numeric) #converts all the data to numeric

names = c('sex', 'cp', 'exang', 'slope', 'thal', 'diagnosis', "restecg", "ca", "fbs") #unsure if ca is a factor
heart_data[,names] <- lapply(heart_data[,names] , factor)

heart_data <- mutate(heart_data, diagnosis = ifelse(diagnosis == "1"|diagnosis == "2"|diagnosis == "3"|diagnosis == "4", "Positive", "Negative"))
heart_data <- mutate(heart_data, diagnosis = as.factor(diagnosis))          #changes the diagnosis values to strings

heart_data



heart_split <- initial_split(heart_data, prop = 0.75, strata = diagnosis)          #splits data
heart_train <- training(heart_split)
heart_test <- testing(heart_split) 

heart_train

train_summary <- heart_train |>
    summarize(across(age:diagnosis, mean, na.rm=TRUE))
train_summary

count_train_summary <- group_by(heart_train, diagnosis) |>
    summarize(count=n())
count_train_summary

table(heart_train$diagnosis, useNA = "ifany")                 # potential way to get the count of each class 
# heart_table = matrix(c(1:70), ncol = 14, byrow=TRUE)
# colnames(heart_table) = c("age","sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", 
#                           "exang", "oldpeak", "slope", "ca", "thal", "diagnosis")
# rownames(heart_table) <- c("mean", "sd", "observations", "min", "max")

# heart_table





heart_test <- heart_train |>
    group_by(diagnosis) |>
    summarize(across(age:thal, mean, na.rm=TRUE))      #Gives table of mean based on diagnosis
heart_test


# heart_test_graph <- heart_train |>
#     ggplot(aes(x = diagnosis, y = thalach)) + #color = diagnosis)) +
#     geom_point()

# heart_test_graph

ERROR: Error in parse(text = x, srcfile = src): <text>:16:52: unexpected '='
15:                      cholesterol = X5,
16:                      fasting_blood_sugar_above120? =
                                                       ^


In [5]:
heart_train

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0,6,Negative
37,1,3,130,250,0,0,187,0,3.5,3,0,3,Negative
41,0,2,130,204,0,2,172,0,1.4,1,0,3,Negative
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
68,1,4,144,193,1,0,141,0,3.4,2,2,7,Positive
57,1,4,130,131,0,0,115,1,1.2,2,1,7,Positive
57,0,2,130,236,0,2,174,0,0.0,2,1,3,Positive


In [7]:
stats_avg <- heart_train |>
    select(age, trestbps, chol, thalach, oldpeak, diagnosis) |>
    group_by(diagnosis) |>
    summarize(across(age:oldpeak, mean, na.rm = TRUE))

stats_avg

diagnosis,age,trestbps,chol,thalach,oldpeak
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Negative,51.82114,128.3008,237.8862,158.9675,0.5699187
Positive,56.58654,132.1442,250.1635,139.1250,1.5788462


In [17]:
sex_count <- heart_train |>
    group_by(diagnosis, sex) |>
    summarize(count = n())
sex_count

cp_count <- heart_train |>
    group_by(diagnosis, cp) |>
    summarize(count = n())
cp_count

fbs_count <- heart_train |>
    group_by(diagnosis, fbs) |>
    summarize(count = n())

restecg_count <- heart_train |>
    group_by(diagnosis, restecg) |>
    summarize(count = n())

exang_count <- heart_train |>
    group_by(diagnosis, exang) |>
    summarize(count = n())

slope_count <- heart_train |>
    group_by(diagnosis, slope) |>
    summarize(count = n())

ca_count <- heart_train |>
    group_by(diagnosis, ca) |>
    summarize(count = n())

thal_count <- heart_train |>
    group_by(diagnosis, thal) |>
    summarize(count = n())


stats_count_factors <- sex_count |>
    rbind(cp_count) |>
    rbind(fbs_count) |>
    rbind(restecg_count) |>
    rbind(exang_count) |>
    rbind(slope_count) |>
    rbind(ca_count) |>
    rbind(thal_count)

stats_count_factors

`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.


diagnosis,sex,count
<fct>,<fct>,<int>
Negative,0,52
Negative,1,71
Positive,0,19
Positive,1,85


`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.


diagnosis,cp,count
<fct>,<fct>,<int>
Negative,1,10
Negative,2,35
Negative,3,49
⋮,⋮,⋮
Positive,2,8
Positive,3,13
Positive,4,78


`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'diagnosis'. You can override using the
`.groups` argument.


diagnosis,sex,count,cp,fbs,restecg,exang,slope,ca,thal
<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Negative,0,52,NA,NA,NA,NA,NA,NA,NA
Negative,1,71,NA,NA,NA,NA,NA,NA,NA
Positive,0,19,NA,NA,NA,NA,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Positive,NA,27,NA,NA,NA,NA,NA,NA,3
Positive,NA,8,NA,NA,NA,NA,NA,NA,6
Positive,NA,69,NA,NA,NA,NA,NA,NA,7
